In [91]:
# randomly sample 5 songs for each user



pprint(user_list)

[{'id': 0,
  'name': 'Genoveva Joyce',
  'tracks': [{'id': '0ZV8Et78hy2FAOel0AADuD',
              'name': 'Help Me Through the Day'},
             {'id': '26rU1B3EydzaYv985RY0k7', 'name': 'A Bhradag Dhubh'},
             {'id': '3eSDuDrj35ojCVBY4qO4U4', 'name': 'Insufferable Days'},
             {'id': '3mnjkR6cXjaR0fSDyMDwyH',
              'name': 'Concierto madrigal (excerpts): Fanfare: Allegro '
                      'marziale'},
             {'id': '4w2l8pTUxsqvgcn6OZdqum', 'name': 'True Union'}]},
 {'id': 1,
  'name': 'Sarah Derby',
  'tracks': [{'id': '7JBjSAR14e65ZQjDv0zLvX', 'name': 'Love Makes the World'},
             {'id': '6q7tIVwfAGc3xsnAWFrJiD',
              'name': 'PLASTIC♡LOVE - House Mix'},
             {'id': '7sLaBmgwiMS8QuVpqF0fPW', 'name': 'Tasnife Eshgh Az Koja'},
             {'id': '2hd6A9jvo0Xsrbwp3CzPaL', 'name': 'Gone Too Long'},
             {'id': '6hvgfdB5UHaPPFicjtiPwV', 'name': 'Paiz'}]},
 {'id': 2,
  'name': 'Teresa Knox',
  'tracks': [{'id': '1YvI